We've included 2 solutions here. There are also many other ways of formatting this data. Don't wory if you haven't done it exactly like we have.... this is just guide and not a bible.

In [1]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Solution using a for loop

First we need to take a look at the JSON so we know what we're dealing with.
- We can view what the JSON contains by checking out the [documentation on the OWM website](https://openweathermap.org/forecast5)
- Or this can be done manually, like below:

In [18]:
city = 'Berlin'
API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

response = requests.get(url)
json = response.json()

json

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1667390400,
   'main': {'temp': 15.19,
    'feels_like': 14.51,
    'temp_min': 15.19,
    'temp_max': 15.45,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 1014,
    'humidity': 67,
    'temp_kf': -0.26},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 6.2, 'deg': 251, 'gust': 10.75},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-11-02 12:00:00'},
  {'dt': 1667401200,
   'main': {'temp': 14.69,
    'feels_like': 13.83,
    'temp_min': 13.68,
    'temp_max': 14.69,
    'pressure': 1013,
    'sea_level': 1013,
    'grnd_level': 1014,
    'humidity': 62,
    'temp_kf': 1.01},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 3},
   'wind': {'speed': 5.62, 'deg': 256, 'gust': 11.28},
   'visibility': 10000,


Now we've discovered what information we have to work with. Let's decide what we want to keep and what we wish to lose.

I feel that from json['list'] it would be good to keep
- 'weather.main', 'weather.description', 'dt_txt', 'main.temp', 'main.feels_like' 'clouds.all', 'rain.3h', 'snow.3h' 'wind.speed', 'wind.deg', 'main.humidity', 'main.pressure'. 

And from json['city'] it would be good to keep 
- 'name' and 'country. 

Just to make sure that we got the right place. And as an added extra we'll also include the time that API call was made, so we know how up to date our forecast is.

**Optional:** let's get a timestamp of when we get the data. Datetime uses the uses the current time of the system, which on local computers is normally correct. But as we're in the cloud, computers are not always in our country, and we therefore add on the timezone module to ensure that our timestamp is local to us and not the computer.

In [3]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

now

datetime.datetime(2022, 11, 2, 10, 41, 53, 817019, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)

**Next** let's loop through the json['list'] information get the weather information

In [5]:
# we'll store the information in this dicitonary:
weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

# let's begin the loop
for i in json['list']:
  weather_dict['city'].append(json['city']['name'])
  weather_dict['country'].append(json['city']['country'])
  weather_dict['forecast_time'].append(i['dt_txt'])
  weather_dict['outlook'].append(i['weather'][0]['main'])
  weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
  weather_dict['temperature'].append(i['main']['temp'])
  weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
  weather_dict['clouds'].append(i['clouds']['all'])
  # sometimes the data is missing for rain and snow. As it is not always raining or snowing
  # we cannot make a DataFrame unless the lists are all the same length, therefore missing values are bad
  # here we say try to append a value if there is one. If not, append a 0
  try:
      weather_dict['rain'].append(i['rain']['3h'])
  except:
      weather_dict['rain'].append('0')
  try:
      weather_dict['snow'].append(i['snow']['3h'])
  except:
      weather_dict['snow'].append('0')
  weather_dict['wind_speed'].append(i['wind']['speed'])
  weather_dict['wind_deg'].append(i['wind']['deg'])
  weather_dict['humidity'].append(i['main']['humidity'])
  weather_dict['pressure'].append(i['main']['pressure'])
  weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
  

**Now** we convert our dictionary to a DataFrame

In [6]:
weather_from_dict_df = pd.DataFrame(weather_dict)

weather_from_dict_df.head()

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-11-02 12:00:00,Clouds,few clouds,13.79,13.00,23,0,0,6.20,251,68,1012,02/11/2022 10:41:53
1,Berlin,DE,2022-11-02 15:00:00,Clouds,few clouds,13.44,12.43,13,0,0,5.62,256,61,1016,02/11/2022 10:41:53
2,Berlin,DE,2022-11-02 18:00:00,Clear,clear sky,11.46,10.25,7,0,0,3.68,252,61,1021,02/11/2022 10:41:53
3,Berlin,DE,2022-11-02 21:00:00,Clear,clear sky,9.89,8.58,2,0,0,2.67,233,67,1022,02/11/2022 10:41:53
4,Berlin,DE,2022-11-03 00:00:00,Clouds,few clouds,8.87,7.66,11,0,0,2.26,186,71,1021,02/11/2022 10:41:53


**As a final step**, to keep everything tidy: let's bring everything we did together in a function. And allow the function to take a list of cities as an input

In [7]:
def get_weather_loop(cities):

  API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

  tz = pytz.timezone('Europe/Berlin')
  now = datetime.now().astimezone(tz)

  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    for i in json['list']:
      weather_dict['city'].append(json['city']['name'])
      weather_dict['country'].append(json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      try:
          weather_dict['rain'].append(i['rain']['3h'])
      except:
          weather_dict['rain'].append('0')
      try:
          weather_dict['snow'].append(i['snow']['3h'])
      except:
          weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  return pd.DataFrame(weather_dict)

In [25]:
get_weather_loop(['Berlin', 'London'])

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-11-02 12:00:00,Clear,clear sky,15.07,14.33,0,0,0,6.20,251,65,1009,02/11/2022 11:55:50
1,Berlin,DE,2022-11-02 15:00:00,Clear,clear sky,14.61,13.72,3,0,0,5.62,256,61,1013,02/11/2022 11:55:50
2,Berlin,DE,2022-11-02 18:00:00,Clear,clear sky,12.66,11.60,5,0,0,3.68,252,62,1017,02/11/2022 11:55:50
3,Berlin,DE,2022-11-02 21:00:00,Clear,clear sky,9.89,8.58,2,0,0,2.67,233,67,1022,02/11/2022 11:55:50
4,Berlin,DE,2022-11-03 00:00:00,Clouds,few clouds,8.87,7.66,11,0,0,2.26,186,71,1021,02/11/2022 11:55:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,London,GB,2022-11-06 21:00:00,Clouds,overcast clouds,12.33,11.60,100,0,0,5.30,217,76,1009,02/11/2022 11:55:50
76,London,GB,2022-11-07 00:00:00,Clouds,overcast clouds,12.65,12.03,100,0,0,5.57,214,79,1009,02/11/2022 11:55:50
77,London,GB,2022-11-07 03:00:00,Clouds,overcast clouds,13.10,12.61,100,0,0,6.40,208,82,1008,02/11/2022 11:55:50
78,London,GB,2022-11-07 06:00:00,Rain,light rain,13.54,13.06,100,0.13,0,6.82,210,81,1008,02/11/2022 11:55:50


# Solution using `.json_normalize()`

**First** we need to make an API call to get our information

In [9]:
city = 'Berlin'
API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
response = requests.get(url)
json = response.json()

If we now try to use `.json_normalize()` on the `list` part of the json, you'll see that json_normalize can get confused when it hits a list instead of a subdictionary. Take a look at the weather column below

In [10]:
pd.DataFrame((json['list'])).head()

,dt,main,weather,clouds,wind,visibility,pop,sys,dt_txt,rain
0,1667390400,"{'temp': 13.79, 'feels_like': 13, 'temp_min': ...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 23},"{'speed': 6.2, 'deg': 251, 'gust': 10.75}",10000,0.0,{'pod': 'd'},2022-11-02 12:00:00,NaN
1,1667401200,"{'temp': 13.44, 'feels_like': 12.43, 'temp_min...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 13},"{'speed': 5.62, 'deg': 256, 'gust': 11.28}",10000,0.0,{'pod': 'd'},2022-11-02 15:00:00,NaN
2,1667412000,"{'temp': 11.46, 'feels_like': 10.25, 'temp_min...","[{'id': 800, 'main': 'Clear', 'description': '...",{'all': 7},"{'speed': 3.68, 'deg': 252, 'gust': 9.36}",10000,0.0,{'pod': 'n'},2022-11-02 18:00:00,NaN
3,1667422800,"{'temp': 9.89, 'feels_like': 8.58, 'temp_min':...","[{'id': 800, 'main': 'Clear', 'description': '...",{'all': 2},"{'speed': 2.67, 'deg': 233, 'gust': 7.17}",10000,0.0,{'pod': 'n'},2022-11-02 21:00:00,NaN
4,1667433600,"{'temp': 8.87, 'feels_like': 7.66, 'temp_min':...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 11},"{'speed': 2.26, 'deg': 186, 'gust': 4.33}",10000,0.0,{'pod': 'n'},2022-11-03 00:00:00,NaN


To fix this we need to flatten the nested list using the parameter `record_path`, and then select the other columns we'd like using the parameter `meta`. At the end of the json_normalize parameters we have `errors='ignore'`, which means that if ever there is a missing value in the JSON, a `NaN` gets inserted into the table. If you'd like more information, please check out the [docs](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html)

In [46]:
pd.json_normalize(json, record_path= ["list", "weather"], errors="ignore")

,id,main,description,icon
0,800,Clear,clear sky,01d
1,800,Clear,clear sky,01d
2,800,Clear,clear sky,01n
3,800,Clear,clear sky,01n
4,801,Clouds,few clouds,02n
5,803,Clouds,broken clouds,04n
6,804,Clouds,overcast clouds,04n
7,804,Clouds,overcast clouds,04d
8,804,Clouds,overcast clouds,04d
9,804,Clouds,overcast clouds,04d


In [35]:
json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']],
                                errors='ignore')

json_norm_df.head()

,id,main,description,icon,dt_txt,main.temp,main.feels_like,clouds.all,rain.3h,snow.3h,wind.speed,wind.deg,main.humidity,main.pressure
0,800,Clear,clear sky,01d,2022-11-02 12:00:00,15.19,14.51,0,NaN,NaN,6.2,251,67,1009
1,800,Clear,clear sky,01d,2022-11-02 15:00:00,14.69,13.83,3,NaN,NaN,5.62,256,62,1013
2,800,Clear,clear sky,01n,2022-11-02 18:00:00,12.7,11.67,5,NaN,NaN,3.68,252,63,1017
3,800,Clear,clear sky,01n,2022-11-02 21:00:00,9.89,8.58,2,NaN,NaN,2.67,233,67,1022
4,801,Clouds,few clouds,02n,2022-11-03 00:00:00,8.87,7.66,11,NaN,NaN,2.26,186,71,1021


Now let's tidy this DataFrame up by
- dropping some columns
- renaming some columns
- adding some columns

In [12]:
# drop id and icon
json_norm_df = json_norm_df.drop(columns=['id', 'icon']).copy()

In [13]:
# rename some columns
json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)

In [14]:
# add some columns
json_norm_df.insert(0, 'city', json['city']['name'])
json_norm_df.insert(1, 'country', json['city']['country'])
json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")

# rearrange the columns
json_norm_df = json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
       'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
       'wind_speed', 'wind_deg', 'humidity', 'pressure',
       'information_retrieved_at']]

json_norm_df.head()

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-11-02 12:00:00,Clouds,few clouds,13.79,13,23,NaN,NaN,6.2,251,68,1012,02/11/2022 10:41:53
1,Berlin,DE,2022-11-02 15:00:00,Clouds,few clouds,13.44,12.43,13,NaN,NaN,5.62,256,61,1016,02/11/2022 10:41:53
2,Berlin,DE,2022-11-02 18:00:00,Clear,clear sky,11.46,10.25,7,NaN,NaN,3.68,252,61,1021,02/11/2022 10:41:53
3,Berlin,DE,2022-11-02 21:00:00,Clear,clear sky,9.89,8.58,2,NaN,NaN,2.67,233,67,1022,02/11/2022 10:41:53
4,Berlin,DE,2022-11-03 00:00:00,Clouds,few clouds,8.87,7.66,11,NaN,NaN,2.26,186,71,1021,02/11/2022 10:41:53


Now we have a very similar DataFrame to what we had with the for loop. Again, let's put this in a function that can take a list of cities

In [15]:
def get_weather_norm(cities):
  
  API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

  df_list = []

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
    response = requests.get(url)
    json = response.json()

    json_norm_df = pd.json_normalize(json['list'], 
                                record_path=['weather'], 
                                meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']], 
                                errors='ignore')
    json_norm_df.drop(columns=['id', 'icon'], inplace=True)
    json_norm_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             'main.feels_like': 'temperature_feels_like',
                             'clouds.all': 'clouds',
                             'rain.3h': 'rain',
                             'snow.3h': 'snow',
                             'wind.speed': 'wind_speed',
                             'wind.deg': 'wind_deg',
                             'main.humidity': 'humidity',
                             'main.pressure': 'pressure',},
                    inplace=True)
    json_norm_df.insert(0, 'city', json['city']['name'])
    json_norm_df.insert(1, 'country', json['city']['country'])
    json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")
    json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
          'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
          'wind_speed', 'wind_deg', 'humidity', 'pressure',
          'information_retrieved_at']]
    df_list.append(json_norm_df)
  return pd.concat(df_list, ignore_index=True)

In [16]:
get_weather_norm(['Berlin', 'London'])

,city,country,outlook,detailed_outlook,forecast_time,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,Clouds,few clouds,2022-11-02 12:00:00,13.79,13,23,NaN,NaN,6.2,251,68,1012,02/11/2022 10:41:53
1,Berlin,DE,Clouds,few clouds,2022-11-02 15:00:00,13.44,12.43,13,NaN,NaN,5.62,256,61,1016,02/11/2022 10:41:53
2,Berlin,DE,Clear,clear sky,2022-11-02 18:00:00,11.46,10.25,7,NaN,NaN,3.68,252,61,1021,02/11/2022 10:41:53
3,Berlin,DE,Clear,clear sky,2022-11-02 21:00:00,9.89,8.58,2,NaN,NaN,2.67,233,67,1022,02/11/2022 10:41:53
4,Berlin,DE,Clouds,few clouds,2022-11-03 00:00:00,8.87,7.66,11,NaN,NaN,2.26,186,71,1021,02/11/2022 10:41:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,London,GB,Clouds,overcast clouds,2022-11-06 21:00:00,12.33,11.6,100,NaN,NaN,5.3,217,76,1009,02/11/2022 10:41:53
76,London,GB,Clouds,overcast clouds,2022-11-07 00:00:00,12.65,12.03,100,NaN,NaN,5.57,214,79,1009,02/11/2022 10:41:53
77,London,GB,Clouds,overcast clouds,2022-11-07 03:00:00,13.1,12.61,100,NaN,NaN,6.4,208,82,1008,02/11/2022 10:41:53
78,London,GB,Rain,light rain,2022-11-07 06:00:00,13.54,13.06,100,0.13,NaN,6.82,210,81,1008,02/11/2022 10:41:53
